# Hands-on Tutorial for Brain Hierarchy Score

This notebook provides a hands-on tutorial for brain hierarchy (BH) score ([Nonaka et al., 2021](https://doi.org/10.1016/j.isci.2021.103013)).
BH score quantifies the hierarchical correspondence between the brain and visual neural network models based on decoding and encoding analyses.
In this tutorial, we will demonstrate how to compute BH score using decoding and encoding accuracies of 29 DNNs.

## Environment setup

Run the following code block to install Python packages required for this notebook.

In [ ]:
!pip install numpy
!pip install scipy
!pip install pandas
!pip install hdf5storage
!pip install matplotlib

Import required modules.

In [2]:
from typing import List, Tuple, Union

import os
from pathlib import Path
import json

import numpy as np
from scipy.stats import spearmanr, t
import pandas as pd
from hdf5storage import loadmat
import matplotlib.pyplot as plt
from pprint import pprint

## Data preparation

Download the dataset used in this notebook.
Please ignore it if you have already downloaded the dataset.

In [ ]:
!wget -O decoding_accuracy.zip https://ndownloader.figshare.com/files/24132263   # Total size: 88.41MB
!wget -O encoding_accuracy.zip https://figshare.com/ndownloader/files/32116400   # Total size: 264.17MB
!unzip -q decoding_accuracy.zip
!unzip -q encoding_accuracy.zip
!mkdir data
!mv decoding_accuracy encoding_accuracy data/
!curl -O https://raw.githubusercontent.com/KamitaniLab/BHscore/master/settings.json

Check the downloaded files.

In [ ]:
!ls data/

Define settings for the analysis.

In [ ]:
# Data settings

# Subjects and ROIs
subjects = ["sub-01", "sub-02", "sub-03"]
rois = ["V1", "V2", "V3", "V4", "HVC"]

# DNNs
with open("settings.json", "r") as f:
    dnns = json.load(f)["dnns"]

decoding_accuracy_dir = Path("data/decoding_accuracy/ImageNetTest")
encoding_accuracy_dir = Path("data/encoding_accuracy/ImageNetTest")

## Loading decoding and encoding prediction accuracies

Load decoding and encoding prediction accuracies from the downloaded data.
BH scores are calculated from the decoding and encoding prediction accuracies.

In [ ]:
decoding_accuracy = pd.DataFrame(columns=["network", "layer", "roi", "accuracy"])
encoding_accuracy = pd.DataFrame(columns=["network", "layer", "roi", "accuracy"])

for dnn_name, dnn in dnns.items():

    layers = dnn['layers']

    dec_acc_array = np.zeros((len(layers), len(rois)))
    enc_acc_array = np.zeros((len(layers), len(rois)))

    for i, lay in enumerate(layers):
        for j, roi in enumerate(rois):

            # Pool decoding accuracies across subjects
            dec_acc_pooled = []
            enc_acc_pooled = []
            for sub in subjects:
                dec_acc_file = decoding_accuracy_dir / dnn['dir'] / lay / sub / roi / 'accuracy.mat'
                enc_acc_file = encoding_accuracy_dir / dnn['dir'] / lay / sub / roi / 'accuracy.mat'

                dec_acc = loadmat(str(dec_acc_file))['accuracy'].ravel()
                enc_acc = loadmat(str(enc_acc_file))['accuracy'].ravel()

                dec_acc_pooled.append(dec_acc)
                enc_acc_pooled.append(enc_acc)

            dec_acc_pooled = np.hstack(dec_acc_pooled)
            enc_acc_pooled = np.hstack(enc_acc_pooled)

            df_dec = pd.DataFrame({
                "network": dnn_name,
                "layer": lay,
                "roi": roi,
                "accuracy": [dec_acc_pooled]
            })
            df_enc = pd.DataFrame({
                "network": dnn_name,
                "layer": lay,
                "roi": roi,
                "accuracy": [enc_acc_pooled]
            })

            decoding_accuracy = pd.concat([decoding_accuracy, df_dec], ignore_index=True)
            encoding_accuracy = pd.concat([encoding_accuracy, df_enc], ignore_index=True)


Check the loaded data.

In [ ]:
decoding_accuracy

In [ ]:
encoding_accuracy

## Calculating BH scores

Define functions to calculate BH scores (original code: https://github.com/KamitaniLab/BHscore).

In [ ]:
# Functions to calculate BH score
# https://github.com/KamitaniLab/BHscore 


def compute_bhscore(
        predacc_list: List[np.ndarray],
        pval: float = 0.05,
        return_tops: bool = False
    ) -> Union[float, Tuple[float, List[np.ndarray]]]:
    """Compute a BH score of a given DNN.

    Parameters
    ----------
    predacc_list : list of arrays
        List of prediction accuracies for a DNN. Each array contains
        prediction accuracies of individual units in a layer, formed as an
         array of ROIs/layers x units/voxels.
    pval : float, default = 0.05
        P-value threshold in unit selection.
    return_tops : bool, default = False
        Returns top ROIs/layers if True.

    Returns
    -------
    bhscore : float
    top_rois: list of arrays
    """

    tops = []
    for predacc in predacc_list:
        # if prediction accuracy is nan, convert it to zero
        predacc[np.isnan(predacc)] = 0

        # for each CNN units, search roi/layer which has the highest prediction accuracy
        pred_max = np.max(predacc, axis=0)
        pred_max_ind = np.argmax(predacc, axis=0)

        # compute p value of the highest decoding accuracy
        tmp = np.sqrt((50 - 2) * (1 - pred_max ** 2))
        tmp = pred_max * tmp
        pvals = 2 * (1 - t.cdf(tmp, df=50 - 2))

        # keep unit with p value < threshold and acc > 0
        threshold = pvals < pval
        plus_unit = pred_max > 0
        select_unit_ind = np.logical_and(threshold, plus_unit)
        pred_max_ind = pred_max_ind[select_unit_ind]

        tops.append(pred_max_ind)

    # get layer numbers of each unit. concatenate best ROIs/layers for all layers
    layer_numbers = []
    tops_flatten = []
    for i_br, br in enumerate(tops):
        layer_numbers.extend(np.repeat(i_br + 1, len(br)))
        tops_flatten.extend(br)

    # compute Spearman's rank correlation
    bhscore, _ = spearmanr(layer_numbers, tops_flatten)

    if return_tops:
        return bhscore, tops
    else:
        return bhscore


def compute_bhscore_layerselect(
        predacc_list: List[np.ndarray],
        pval: float = 0.05,
        n_layers: int = 5,
        n_repeat: int = 100,
        return_top_rois: bool = False
    ) -> Union[float, Tuple[float, List[List[np.ndarray]]]]:
    """Compute a BH score of a given DNN, random layer selection version.

    Parameters
    ----------
    predacc_list : list of arrays
        List of prediction accuracies for a DNN. Each array contains
        prediction accuracies of individual units in a layer, formed as an
         array of ROIs x units.
    pval : float, default = 0.05
        P-value threshold in unit selection.
    n_layers : int, default = 5
        The number of layers used to compute the BH score. Note that the first
        and last layers are always included in the computation. Thus,
        (n_layers - 2) layers are randomly selected from the representative
        layers except the first and last ones.
    n_repeat : int, default = 100
        The number of random layer selection.
    return_top_rois : bool, default = False
        Returns top ROIs if True.

    Returns
    -------
    bhscore_list : arary of float
    top_rois_list : list of list of arrays
    """

    bhscore_list = np.zeros(n_repeat)
    top_rois_list = []
    for i_s in range(n_repeat):
        # sample layers
        sample_index = np.random.choice(np.arange(1, len(predacc_list)-1), size=n_layers - 2, replace=False)
        sample_index = np.sort(sample_index)
        predacc_list_sampled = [predacc_list[0]] + [predacc_list[i] for i in sample_index] + [predacc_list[-1]]

        bhscore, top_rois = compute_bhscore(predacc_list_sampled, pval, return_tops=True)
        bhscore_list[i_s] = bhscore
        top_rois_list.append(top_rois)

    if return_top_rois:
        return bhscore_list, top_rois_list
    else:
        return bhscore_list


Ccalculate BH scores of each DNN using the function defined above.

In [ ]:
bhscore_decoding = {}  # Decoding-based BH scores
bhscore_encoding = {}  # Encoding-based BH scores
bhscore = {}           # Average BH scores

decoding_top_rois = {}
encoding_top_layers = {}

for dnn_name, dnn in dnns.items():

    layers = dnn['layers']

    # Decoding-based BH score
    dec_pred_acc = []
    for i, lay in enumerate(layers):
        df = np.stack([
            list(decoding_accuracy.query(f"network == '{dnn_name}' & layer == '{lay}' & roi == '{roi}'")["accuracy"])[0]
            for roi in rois
        ])
        dec_pred_acc.append(df)
    #bhscore_dec, top_rois = compute_bhscore(dec_pred_acc, return_tops=True)
    bhscore_dec = compute_bhscore_layerselect(dec_pred_acc, n_layers=5, n_repeat=100)
    bhscore_dec = np.mean(bhscore_dec)

    # Encoding-based BH score
    enc_pred_acc = []
    for i, roi in enumerate(rois):
        df = np.stack([
            list(encoding_accuracy.query(f"network == '{dnn_name}' & layer == '{lay}' & roi == '{roi}'")["accuracy"])[0]
            for lay in layers
        ])
        enc_pred_acc.append(df)
    bhscore_enc = compute_bhscore(enc_pred_acc)

    bhscore_decoding[dnn_name] = bhscore_dec
    bhscore_encoding[dnn_name] = bhscore_enc
    bhscore[dnn_name] = np.mean([bhscore_dec, bhscore_enc])


Check the results.

In [ ]:
bhscore

In [ ]:
bhscore_decoding

In [ ]:
bhscore_encoding

## Displaying BH scores

Display the ranking of BH scores for the 29 DNNs (Figure 3 in [the original paper](https://doi.org/10.1016/j.isci.2021.103013)).
Note that the results may slightly differ from the original paper due to some randomness in the calculation of BH scores.

In [ ]:
# Display BH score ranking
nets = np.array([n for n in bhscore.keys()])
bhscores = np.array([s for s in bhscore.values()])

ranking_index = np.argsort(bhscores)[::-1]
nets = nets[ranking_index]
bhscores = bhscores[ranking_index]

for net, score in zip(nets, bhscores):
    print('{}: {}'.format(net, score))

# Bar chart
fig = plt.figure(figsize=(8, 14))

ypos = range(nets.shape[0])[::-1]

plt.barh(ypos, bhscores)

plt.title('BH score')

plt.yticks(ypos, nets)
plt.ylim([-0.5, nets.shape[0] - 0.5])

for yp, bhs in zip(ypos, bhscores):
    plt.text(
        bhs - 0.005, yp, '%.2f' % bhs, color='white',
        horizontalalignment='right',
        verticalalignment='center'
    )

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.xticks([])

plt.show()

## Distribution of most preditive ROIs/layers

Here we plot the distribution of the most predictive ROIs/layers for each DNN.

Define a function to plot the distribution of the most predictive ROIs/layers.

In [ ]:
def makefig_top_distribution(accuracy: List[np.ndarray], layers: List[str], rois: List[str], dnn_name: str, type: str = "decoding"):
    """Make a figure of the distribution of top ROIs/layers.

    Parameters
    ----------
    accuracy : list of arrays
        List of prediction accuracies for a DNN.
    layers : list of str
        List of layer names.
    rois : list of str
        List of ROI names.
    dnn_name : str
        DNN name.
    type : str, default = "decoding"
        Type of analysis. "decoding" or "encoding".

    Returns
    -------
    fig : matplotlib.figure.Figure
    """

    bh_score, tops = compute_bhscore(accuracy, pval=0.05, return_tops=True)

    n_layers = len(layers)
    n_rois = len(rois)

    if type == "decoding":
        components = layers
        n_components = len(components)
        features = rois
    elif type == "encoding":
        components = rois
        n_components = len(components)
        features = layers

    fig = plt.figure(figsize=(4, 1 * n_components))

    for i, (comp, top) in enumerate(zip(components, tops)):
        y = np.array([
            np.sum(top == i)
            for i, _ in enumerate(features)
        ])
        y = y / np.size(top)

        x = np.arange(len(features))

        plt.subplot(n_components, 1, n_components - i)

        plt.bar(x, y, width=0.8, color='gray')
        plt.xticks([])
        if i == 0:
            plt.xticks(x, labels=features, fontsize=9)
        plt.ylim([0, 1])
        plt.yticks([])
        plt.text(-0.5, 0.75, comp, fontsize=12)

        if i == n_components - 1:
            plt.title('{} {} BH score = {:.2f}'.format(dnn_name, type, bh_score), fontsize=12)

        # Box off
        ax = plt.gca()
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_ticks_position('left')

    return fig


Plot the distribution of the most predictive ROIs/layers for "AlexNet".

In [ ]:
network = "AlexNet"

layers = dnns[network]['layers']
rois = ["V1", "V2", "V3", "V4", "HVC"]

# Decoding-based BH score
dec_pred_acc = []
for i, lay in enumerate(layers):
    acc = np.stack([
        list(decoding_accuracy.query(f"network == '{dnn_name}' & layer == '{lay}' & roi == '{roi}'")["accuracy"])[0]
        for roi in rois
    ])
    dec_pred_acc.append(acc)

makefig_top_distribution(dec_pred_acc, dnn['layers'], rois, dnn_name, type="decoding")

# Encoding-based BH score
enc_pred_acc = []
for i, roi in enumerate(rois):
    acc = np.stack([
        list(encoding_accuracy.query(f"network == '{dnn_name}' & layer == '{lay}' & roi == '{roi}'")["accuracy"])[0]
        for lay in layers
    ])
    enc_pred_acc.append(acc)
bhscore_enc = compute_bhscore(enc_pred_acc)

makefig_top_distribution(enc_pred_acc, dnn['layers'], rois, dnn_name, type="encoding")

plt.show()


Plot the distribution of the most predictive ROIs/layers for all DNNs.

In [ ]:
rois = ["V1", "V2", "V3", "V4", "HVC"]

for dnn_name, dnn in dnns.items():

    layers = dnn['layers']

    # Decoding-based BH score
    dec_pred_acc = []
    for i, lay in enumerate(layers):
        acc = np.stack([
            list(decoding_accuracy.query(f"network == '{dnn_name}' & layer == '{lay}' & roi == '{roi}'")["accuracy"])[0]
            for roi in rois
        ])
        dec_pred_acc.append(acc)

    makefig_top_distribution(dec_pred_acc, dnn['layers'], rois, dnn_name, type="decoding")

    # Encoding-based BH score
    enc_pred_acc = []
    for i, roi in enumerate(rois):
        acc = np.stack([
            list(encoding_accuracy.query(f"network == '{dnn_name}' & layer == '{lay}' & roi == '{roi}'")["accuracy"])[0]
            for lay in layers
        ])
        enc_pred_acc.append(acc)
    bhscore_enc = compute_bhscore(enc_pred_acc)

    makefig_top_distribution(enc_pred_acc, dnn['layers'], rois, dnn_name, type="encoding")

    break
